In [1]:
# This macro helps optimize the innermost loop in the kernel on your machine. You should not need to use it anywhere else
using LoopVectorization: @turbo

# This function takes in matrices ABC and adds B times C to the matrix A
function add_to_A_B_times_C!(A, B, C)
    @turbo for j in axes(C, 2)
        for k in axes(B, 2)
            for i in axes(A, 1)
                A[i, j] += B[i, k] * C[k, j]
            end
        end
    end
end

add_to_A_B_times_C! (generic function with 1 method)

In [2]:
# This function takes in matrices ABC and adds B times C to the matrix A
# It uses blocking into blocks of size bks
function add_to_A_B_times_C!(A, B, C, bks)
    for i in 1:bks:size(A, 1)
        for j in 1:bks:size(C, 2)
            for k in 1:bks:size(B, 2)
                i_block_end = min(i+bks-1, size(A, 1))
                j_block_end = min(j+bks-1, size(C, 2))
                k_block_end = min(k+bks-1, size(B, 2))

                # Extracting the blocks
                A_block = view(A, i:i_block_end, j:j_block_end)
                B_block = view(B, i:i_block_end, k:k_block_end)
                C_block = view(C, k:k_block_end, j:j_block_end)

                # Perform block multiplication
                add_to_A_B_times_C!(A_block, B_block, C_block)
            end
        end
    end
end


add_to_A_B_times_C! (generic function with 2 methods)

In [3]:
import Pkg.instantiate
instantiate()
using BenchmarkTools: @btime, @belapsed, @ballocated
using LinearAlgebra: mul!
using CairoMakie

In [4]:
function oblivious_add_to_A_B_times_C!(A, B, C, bks)
    i_size, j_size, k_size = size(A, 1), size(C, 2), size(B, 2)

    # Handle small matrices where block size is larger than matrix dimensions
    if min(i_size, j_size, k_size) <= bks
        add_to_A_B_times_C!(A, B, C)
        return
    end

    i_mid = div(i_size, 2)
    j_mid = div(j_size, 2)
    k_mid = div(k_size, 2)

    for i1 in [1, i_mid + 1]
        for j1 in [1, j_mid + 1]
            for k1 in [1, k_mid + 1]
                i2, j2, k2 = min(i1 + i_mid - 1, i_size), min(j1 + j_mid - 1, j_size), min(k1 + k_mid - 1, k_size)

                if min(i2 - i1 + 1, j2 - j1 + 1, k2 - k1 + 1) <= bks
                    add_to_A_B_times_C!(view(A, i1:i2, j1:j2), view(B, i1:i2, k1:k2), view(C, k1:k2, j1:j2))
                else
                    oblivious_add_to_A_B_times_C!(view(A, i1:i2, j1:j2), view(B, i1:i2, k1:k2), view(C, k1:k2, j1:j2), bks)
                end
            end
        end
    end
end


oblivious_add_to_A_B_times_C! (generic function with 1 method)

In [12]:
using BenchmarkTools
using CairoMakie

include("/Users/G/Downloads/HW2/HW2_code/HW2_your_code.jl")

l, m, n = 500, 500, 500
block_sizes = [10, 20, 30, 40, 50]

time_oblivious = Float64[]

for bks in block_sizes
    A = zeros(l, n)
    B = randn(l, m)
    C = randn(m, n)
    push!(time_oblivious, @elapsed oblivious_add_to_A_B_times_C!(A, B, C, bks))
end
pl_oblivious = scatter(block_sizes, time_oblivious, label="oblivious", axis=(xlabel="block size", ylabel="time", yscale=log10))
axislegend(position=:lt)
save("performance_blocked_and_oblivious.pdf", pl_oblivious)


CairoMakie.Screen{PDF}
